In [21]:
import re
import json
import time
from googletrans import Translator

In [24]:
dataset = json.load(open('data/Psychology-10K.json'))

In [5]:
def translate_batch(data_batch, translator, dest_lang):
    translations_instruction = translator.translate(
        [data['instruction'] for data in data_batch],
        src='en',
        dest=dest_lang
    )
    
    translations_input = translator.translate(
        [data['input'] for data in data_batch],
        src='en',
        dest=dest_lang
    )
    
    translations_output = translator.translate(
        [data['output'] for data in data_batch],
        src='en',
        dest=dest_lang
    )
    
    for i, data in enumerate(data_batch):
        data['instruction'] = translations_instruction[i].text
        data['input'] = translations_input[i].text
        data['output'] = translations_output[i].text
    return data_batch

In [31]:
translator = Translator()

batch_size = 5
dataset_translated = []
batch = []

for index, data in enumerate(dataset):
    batch.append(data)

    if len(batch) >= batch_size or index == len(dataset) - 1:
        while True:
            try:
                batch_translated = []

                for batch_data in batch:
                    data_translated = {}

                    for key, value in batch_data.items():
                        placeholders = re.findall(r'{.*?}', value)
                        cleaned_value = re.sub(r'{.*?}', '', value)

                        translation = translator.translate(cleaned_value, dest='id')
                        translated_value = translation.text

                        for placeholder in placeholders:
                            translated_value += ' ' + placeholder

                        data_translated[key] = translated_value.strip()

                    batch_translated.append(data_translated)

                dataset_translated.extend(batch_translated)
                batch = []

                time.sleep(5)
                print(f"Translated {index + 1} out of {len(dataset)}")

                break  # Break out of the retry loop on success

            except Exception as e:
                print(f"Error occurred: {e}")
                print("Retrying after 30 seconds...")
                time.sleep(30)  # Wait for 5 seconds before retrying

Translated 5 out of 9846
Translated 10 out of 9846
Translated 15 out of 9846
Translated 20 out of 9846
Translated 25 out of 9846
Translated 30 out of 9846
Translated 35 out of 9846
Translated 40 out of 9846
Translated 45 out of 9846
Translated 50 out of 9846
Translated 55 out of 9846
Translated 60 out of 9846
Translated 65 out of 9846
Translated 70 out of 9846
Translated 75 out of 9846
Translated 80 out of 9846
Translated 85 out of 9846
Translated 90 out of 9846
Translated 95 out of 9846
Translated 100 out of 9846
Translated 105 out of 9846
Translated 110 out of 9846
Translated 115 out of 9846
Translated 120 out of 9846
Translated 125 out of 9846
Translated 130 out of 9846
Translated 135 out of 9846
Translated 140 out of 9846
Translated 145 out of 9846
Translated 150 out of 9846
Translated 155 out of 9846
Translated 160 out of 9846
Translated 165 out of 9846
Translated 170 out of 9846
Translated 175 out of 9846
Translated 180 out of 9846
Translated 185 out of 9846
Translated 190 out of

In [32]:
with open('Psychology-10K-Indo.json', "w", encoding='utf-8') as f:
    json.dump(dataset_translated, f, ensure_ascii=False, indent=4)

print("JSON Succesfully Exported.")

JSON Succesfully Exported.
